# Making music with Magenta

[Magenta](https://magenta.tensorflow.org/) is a Python library that helps you generate art and music. In this tutorial, we'll talk about the music generation bits in `note_seq` -- how to make your browser sing, and in particular, how to make your browser sing like you!

As a library, `note_seq` can help you:
- make music using some of the neat abstractions and utilities in the library
- use Machine Learning models to generate music.

# Basic Instructions

1. Double click on the hidden cells to make them visible, or select "View > Expand Sections" in the menu at the top.
2. Hover over the "`[ ]`" in the top-left corner of each cell and click on the "Play" button to run it, in order.
3. Listen to the generated samples.
4. Make it your own: copy the notebook, modify the code, train your own models, upload your own MIDI, etc. See the Magenta code on [GitHub](https://github.com/magenta/magenta) for more information!

# Step 0: First things first!
If you're going to use `Magenta`, you need to install it and its dependencies. Some of the later examples will also download other dependencies (such as models and checkpoints)

In [1]:
#@test {"output": "ignore"}
print('Installing dependencies...')
!apt-get update -qq && apt-get install -qq libfluidsynth1 fluid-soundfont-gm build-essential libasound2-dev libjack-dev
!pip install -qU pyfluidsynth pretty_midi

!pip install -qU magenta

# Hack to allow python to pick up the newly-installed fluidsynth lib. 
# This is only needed for the hosted Colab environment.
import ctypes.util
orig_ctypes_util_find_library = ctypes.util.find_library
def proxy_find_library(lib):
  if lib == 'fluidsynth':
    return 'libfluidsynth.so.1'
  else:
    return orig_ctypes_util_find_library(lib)
ctypes.util.find_library = proxy_find_library

print('Importing libraries and defining some helper functions...')
from google.colab import files

import magenta
import note_seq
import tensorflow

print('🎉 Done!')
print(magenta.__version__)
print(tensorflow.__version__)

Installing dependencies...


'apt-get' is not recognized as an internal or external command,
operable program or batch file.


Importing libraries and defining some helper functions...


  ERROR: Command errored out with exit status 1:
   command: 'C:\Users\Mukund\anaconda3\python.exe' -u -c 'import io, os, sys, setuptools, tokenize; sys.argv[0] = '"'"'C:\\Users\\Mukund\\AppData\\Local\\Temp\\pip-install-2xprfdtc\\llvmlite_8201dbc65469421d808432b2253f3676\\setup.py'"'"'; __file__='"'"'C:\\Users\\Mukund\\AppData\\Local\\Temp\\pip-install-2xprfdtc\\llvmlite_8201dbc65469421d808432b2253f3676\\setup.py'"'"';f = getattr(tokenize, '"'"'open'"'"', open)(__file__) if os.path.exists(__file__) else io.StringIO('"'"'from setuptools import setup; setup()'"'"');code = f.read().replace('"'"'\r\n'"'"', '"'"'\n'"'"');f.close();exec(compile(code, __file__, '"'"'exec'"'"'))' bdist_wheel -d 'C:\Users\Mukund\AppData\Local\Temp\pip-wheel-grn8q13r'
       cwd: C:\Users\Mukund\AppData\Local\Temp\pip-install-2xprfdtc\llvmlite_8201dbc65469421d808432b2253f3676\
  Complete output (24 lines):
  running bdist_wheel
  C:\Users\Mukund\anaconda3\python.exe C:\Users\Mukund\AppData\Local\Temp\pip-instal

ModuleNotFoundError: No module named 'google.colab'

# Step 1. Making sounds with NoteSequences

Everything in `Magenta` is centered around [NoteSequences](https://github.com/magenta/note-seq/blob/main/note_seq/protobuf/music.proto#L27). This is an abstract representation of a series of notes, each with different pitches, instruments and strike velocities, much like [MIDI](https://en.wikipedia.org/wiki/MIDI).

For example, this is a `NoteSequence` that represents "Twinkle Twinkle Little Star". Try changing the pitches to see how the sound changes!

In [ ]:
from note_seq.protobuf import music_pb2

twinkle_twinkle = music_pb2.NoteSequence()

# Add the notes to the sequence.
twinkle_twinkle.notes.add(pitch=60, start_time=0.0, end_time=0.5, velocity=80)
twinkle_twinkle.notes.add(pitch=60, start_time=0.5, end_time=1.0, velocity=80)
twinkle_twinkle.notes.add(pitch=67, start_time=1.0, end_time=1.5, velocity=80)
twinkle_twinkle.notes.add(pitch=67, start_time=1.5, end_time=2.0, velocity=80)
twinkle_twinkle.notes.add(pitch=69, start_time=2.0, end_time=2.5, velocity=80)
twinkle_twinkle.notes.add(pitch=69, start_time=2.5, end_time=3.0, velocity=80)
twinkle_twinkle.notes.add(pitch=67, start_time=3.0, end_time=4.0, velocity=80)
twinkle_twinkle.notes.add(pitch=65, start_time=4.0, end_time=4.5, velocity=80)
twinkle_twinkle.notes.add(pitch=65, start_time=4.5, end_time=5.0, velocity=80)
twinkle_twinkle.notes.add(pitch=64, start_time=5.0, end_time=5.5, velocity=80)
twinkle_twinkle.notes.add(pitch=64, start_time=5.5, end_time=6.0, velocity=80)
twinkle_twinkle.notes.add(pitch=62, start_time=6.0, end_time=6.5, velocity=80)
twinkle_twinkle.notes.add(pitch=62, start_time=6.5, end_time=7.0, velocity=80)
twinkle_twinkle.notes.add(pitch=60, start_time=7.0, end_time=8.0, velocity=80) 
twinkle_twinkle.total_time = 8

twinkle_twinkle.tempos.add(qpm=60);

# This is a colab utility method that visualizes a NoteSequence.
note_seq.plot_sequence(twinkle_twinkle)

# This is a colab utility method that plays a NoteSequence.
note_seq.play_sequence(twinkle_twinkle,synth=note_seq.fluidsynth)

# Here's another NoteSequence!
teapot = music_pb2.NoteSequence()
teapot.notes.add(pitch=69, start_time=0, end_time=0.5, velocity=80)
teapot.notes.add(pitch=71, start_time=0.5, end_time=1, velocity=80)
teapot.notes.add(pitch=73, start_time=1, end_time=1.5, velocity=80)
teapot.notes.add(pitch=74, start_time=1.5, end_time=2, velocity=80)
teapot.notes.add(pitch=76, start_time=2, end_time=2.5, velocity=80)
teapot.notes.add(pitch=81, start_time=3, end_time=4, velocity=80)
teapot.notes.add(pitch=78, start_time=4, end_time=5, velocity=80)
teapot.notes.add(pitch=81, start_time=5, end_time=6, velocity=80)
teapot.notes.add(pitch=76, start_time=6, end_time=8, velocity=80)
teapot.total_time = 8

teapot.tempos.add(qpm=60);

note_seq.plot_sequence(teapot)
note_seq.play_sequence(teapot,synth=note_seq.synthesize)

You can use other instruments for your sequences. For example, the sequence below should sound like a drum solo!

In [ ]:
drums = music_pb2.NoteSequence()

drums.notes.add(pitch=36, start_time=0, end_time=0.125, is_drum=True, instrument=10, velocity=80)
drums.notes.add(pitch=38, start_time=0, end_time=0.125, is_drum=True, instrument=10, velocity=80)
drums.notes.add(pitch=42, start_time=0, end_time=0.125, is_drum=True, instrument=10, velocity=80)
drums.notes.add(pitch=46, start_time=0, end_time=0.125, is_drum=True, instrument=10, velocity=80)
drums.notes.add(pitch=42, start_time=0.25, end_time=0.375, is_drum=True, instrument=10, velocity=80)
drums.notes.add(pitch=42, start_time=0.375, end_time=0.5, is_drum=True, instrument=10, velocity=80)
drums.notes.add(pitch=42, start_time=0.5, end_time=0.625, is_drum=True, instrument=10, velocity=80)
drums.notes.add(pitch=50, start_time=0.5, end_time=0.625, is_drum=True, instrument=10, velocity=80)
drums.notes.add(pitch=36, start_time=0.75, end_time=0.875, is_drum=True, instrument=10, velocity=80)
drums.notes.add(pitch=38, start_time=0.75, end_time=0.875, is_drum=True, instrument=10, velocity=80)
drums.notes.add(pitch=42, start_time=0.75, end_time=0.875, is_drum=True, instrument=10, velocity=80)
drums.notes.add(pitch=45, start_time=0.75, end_time=0.875, is_drum=True, instrument=10, velocity=80)
drums.notes.add(pitch=36, start_time=1, end_time=1.125, is_drum=True, instrument=10, velocity=80)
drums.notes.add(pitch=42, start_time=1, end_time=1.125, is_drum=True, instrument=10, velocity=80)
drums.notes.add(pitch=46, start_time=1, end_time=1.125, is_drum=True, instrument=10, velocity=80)
drums.notes.add(pitch=42, start_time=1.25, end_time=1.375, is_drum=True, instrument=10, velocity=80)
drums.notes.add(pitch=48, start_time=1.25, end_time=1.375, is_drum=True, instrument=10, velocity=80)
drums.notes.add(pitch=50, start_time=1.25, end_time=1.375, is_drum=True, instrument=10, velocity=80)
drums.total_time = 1.375

drums.tempos.add(qpm=60)

# This is a colab utility method that visualizes a NoteSequence.
note_seq.plot_sequence(drums)

# This is a colab utility method that plays a NoteSequence.
note_seq.play_sequence(drums,synth=note_seq.fluidsynth)

In [ ]:
import magenta
import note_seq
import tensorflow
from note_seq.protobuf import music_pb2

# set starting pitch note on piano keyboard, you can adjust as per raga needs
pitch = 60 # C5
# 12 notes on piano keyboard
sa,ri1,ri2, ga1,ga2, ma1, ma2, pa, da1,da2, ni1,ni2, sam = pitch, pitch+1, pitch+2, pitch+3, pitch+4, pitch+5,pitch+6,pitch+7,pitch+8,pitch+9,pitch+10,pitch+11,pitch+12
raga_notes = [sa, ri1, ga2, ma1, pa, da1, ni2, sam]
def play_raga(raga_notes):
  raga = music_pb2.NoteSequence()
  s_time = 0.0
# aarohana
  for swara in raga_notes:
    raga.notes.add(pitch=swara, start_time=s_time, end_time=s_time+0.5, velocity=80 )
    s_time += 0.5
  # avarohana
  for swara in raga_notes[::-1]:  # reverse notes and play as the number of notes are same as aarohana
    raga.notes.add(pitch=swara, start_time=s_time, end_time=s_time+0.5, velocity=80 )
    s_time += 0.5
  return raga 

In [ ]:
# Example raga notes
# Mayamalavagowla ragam
mg = play_raga(raga_notes)
mg.total_time = 16
mg.tempos.add(qpm=60);
# This is a colab utility method that visualizes a NoteSequence.
note_seq.plot_sequence(mg)
# This is a colab utility method that plays a NoteSequence.
note_seq.play_sequence(mg,synth=note_seq.fluidsynth)
print("Mayamalavagowla [sa, ri1, ga2, ma1, pa, da1, ni2, sam] -[C,Db,E,F,G,Ab,B,C']")

In [ ]:
# Shankarabharanam ragam
raga_notes = [sa, ri2, ga2, ma1, pa, da2, ni2, sam]
sb =play_raga(raga_notes)
sb.total_time = 16
sb.tempos.add(qpm=60);
note_seq.plot_sequence(sb)
note_seq.play_sequence(sb,synth=note_seq.fluidsynth)
print("Sankarabharanam [sa, ri2, ga2, ma1, pa, da2, ni2, sam] - [C,D,E,F,G,A,B,C']")

In [ ]:
# Kalyani ragam
raga_notes = [sa, ri2, ga2, ma2, pa, da2, ni2, sam]
kl =play_raga(raga_notes)
kl.total_time = 16
kl.tempos.add(qpm=60);
note_seq.plot_sequence(kl)
note_seq.play_sequence(kl,synth=note_seq.fluidsynth)
print("Kalyani [sa, ri2, ga2, ma2, pa, da2, ni2, sam]-[C,D,E,F#,G,A,B,C']")

In [ ]:
# basic practice of raga swaras (notes)
import magenta
import note_seq
import tensorflow

from note_seq.protobuf import music_pb2

def play_sarali_swara(raga_notes):
    raga = music_pb2.NoteSequence() # returns the Note Sequence of a raga
    print(raga)
    s_time = 0.0
    # set increment to 0.6 for 1st kalam (slow pace-1 note at a time),
    # 0.3 for 2nd kalam (medium pace- 2 notes at a time) and
    # 0.2 for 3rd kalam (fast pace- 4 notes at a time)
    increment = 0.2
    vel = 50
    #arohana
    # 1234-32-1232-1234 pattern
    for s in range(2): # 1st loop for aarohana(ascending notes) and 2nd loop for avarohana(descending notes)
        if s == 1:
        # reverse notes for avarohana
            raga_notes = raga_notes[::-1]
        for i in range(5):
            for j in range(i,i+4):
                raga.notes.add(pitch=raga_notes[j], start_time=s_time, end_time=s_time+increment, velocity=vel )
                s_time += increment
            for k in range(i+2,i,-1):
                raga.notes.add(pitch=raga_notes[k], start_time=s_time, end_time=s_time+increment, velocity=vel )
                s_time += increment
            for k in range(i,i+2):
                raga.notes.add(pitch=raga_notes[k], start_time=s_time, end_time=s_time+increment, velocity=vel )
                s_time += increment
            for k in range(i+2,i,-1):
                raga.notes.add(pitch=raga_notes[k], start_time=s_time, end_time=s_time+increment, velocity=vel )
                s_time += increment
            for j in range(i,i+4):
                raga.notes.add(pitch=raga_notes[j], start_time=s_time, end_time=s_time+increment, velocity=vel )
                s_time += increment
#return Note sequence to play
    return raga

In [ ]:
# Example raga notes
# sarali varasai
raga_notes = [sa, ri1, ga2, ma1, pa, da1, ni2, sam]
sv = play_sarali_swara(raga_notes)
#sv.total_time = 16
#sv.tempos.add(qpm=60);
# This is a colab utility method that visualizes a NoteSequence.
note_seq.plot_sequence(sv)
# This is a colab utility method that plays a NoteSequence.
note_seq.play_sequence(sv,synth=note_seq.fluidsynth)
#print("Mayamalavagowla [sa, ri1, ga2, ma1, pa, da1, ni2, sam] -[C,Db,E,F,G,Ab,B,C']")

In [ ]:
from note_seq.protobuf import music_pb2
# Note sequence code for sri gananadha pillari geetam (used for basic training in Carnatic Music)
# Demostrates variable length notes play
# set starting pitch note on piano keyboard
pitch= 60 # C5
# 12 (7 white and 5 black keys) notes on piano keyboard octave
sa,ri1,ri2, ga1,ga2, ma1, ma2, pa, da1,da2, ni1,ni2 = pitch, pitch+1, pitch+2, pitch+3, pitch+4, pitch+5,pitch+6,pitch+7,pitch+8,pitch+9,pitch+10,pitch+11
sam, rim1 = pitch+12, pitch+13 # sa, ri1 of next octave
#malahari ragam 15th Melakarta Mayamalavagowla janya (child ragam) 5 in aarohana, 6 in avarohana, ni omitted
#aarohana_notes = [sa, ri1, ma1, pa, da1, sam] # only 5 notes in aarohana (ascending notes)
#avarohana_notes = [sam, da1, pa, ma1, ga2, ri1, sa] # 6 notes in avarohana (descending notes)
#roopaka talam with 6 beats pattern
srigananadha_notes= [ma1, pa, da1, sam, sam, rim1, rim1, sam, da1, pa, ma1, pa ]
srigananadha_notes.extend([ri1, ma1, pa, da1, ma1, pa, da1, pa, ma1, ga2, ri1, sa])
srigananadha_notes.extend([sa, ri1, ma1, ga2, ri1, sa, ri1, ga2, ri1, sa]) #first ga2 and last sa need to be continued twice the length of other notes
srigananadha_notes.extend([ri1, ma1, pa, da1, ma1, pa, da1, pa, ma1, ga2, ri1, sa])
def add_note_length(raga_notes):
    length_of_notes = 1
    randomlist = []
    for swara in raga_notes:
        randomlist.append((swara, length_of_notes))
    return randomlist
#sri_gananadha_notes_with_length = add_note_length(srigananadha_notes)
# use above add_notes_length() to generate note, length tuples and set variable lengths manually, as required
sri_gananadha_notes_with_length = [(65, 1), (67, 1), (68, 1), (72, 1), (72, 1), (73, 1), (73, 1), (72, 1), (68, 1), (67, 1), (65, 1), (67, 1)]
sri_gananadha_notes_with_length.extend([(61, 1), (65, 1), (67, 1), (68, 1), (65, 1), (67, 1), (68, 1), (67, 1), (65, 1), (64, 1), (61, 1), (60, 1)])
#changed lengths to 3rd and last tuples below
sri_gananadha_notes_with_length.extend([(60, 1), (61, 1), (65, 2), (64, 1), (61, 1), (60, 1), (61, 1), (64, 1), (61, 1), (60, 2)])
sri_gananadha_notes_with_length.extend([(61, 1), (65, 1), (67, 1), (68, 1), (65, 1), (67, 1), (68, 1), (67, 1), (65, 1), (64, 1), (61, 1), (60, 1)])
def play_srigananadha(raga_notes):
    raga = music_pb2.NoteSequence()
    s_time = 0.0
    for swara in raga_notes:
        raga.notes.add(pitch=swara[0], start_time=s_time, end_time=s_time+(0.5*swara[1]), velocity=80 )
        s_time += 0.5*swara[1]
    return raga
 

In [ ]:
malahari = play_srigananadha(sri_gananadha_notes_with_length)
#print(malahari)
malahari.total_time = 48
malahari.tempos.add(qpm=60);
note_seq.plot_sequence(malahari)
note_seq.play_sequence(malahari,synth=note_seq.fluidsynth) 

In [ ]:
import random
# set starting scale note on piano keyboard
pitch= 60 # C5
# 12 (7 white and 5 black keys) notes on piano keyboard octave
sa,ri1,ri2, ga1,ga2, ma1, ma2, pa, da1,da2, ni1,ni2 = pitch, pitch+1, pitch+2, pitch+3, pitch+4, pitch+5,pitch+6,pitch+7,pitch+8,pitch+9,pitch+10,pitch+11
sam, rim1 = pitch+12, pitch+13 # sa, ri1 of next octave
# Mayamalavagowla raga notes to be used for random note generation from within the raga notes
raga_notes = [sa, ri1, ga2, ma1, pa, da1, ni2, sam]
def generate_random_notes(raga_notes):
    raga = music_pb2.NoteSequence()
    no_of_notes = 10 # no of random notes to be generated
    max_length_of_notes = 2 # length of each note to be played
    randomlist = []
    for i in range(0,no_of_notes):
        n = raga_notes[random.randint(0,7)] # generate random note in the given raga
        t = random.randint(1,max_length_of_notes) # generate random length to play note
        randomlist.append((n,t))
    return randomlist
def play_random_notes(raga_notes):
    raga = music_pb2.NoteSequence()
    s_time = 0.0
    for swara in raga_notes:
        raga.notes.add(pitch=swara[0], start_time=s_time, end_time=s_time+(0.5*swara[1]), velocity=80 )
        s_time += 0.5*swara[1]
    return raga


In [ ]:
#send this to NodeSequence to generate raga notes
random_notes = generate_random_notes(raga_notes)
print(random_notes)

In [ ]:
random_notes_play = play_random_notes(random_notes)
random_notes_play.total_time = 100
random_notes_play.tempos.add(qpm=60);
note_seq.plot_sequence(random_notes_play)
note_seq.play_sequence(random_notes_play,synth=note_seq.fluidsynth)

In [ ]:
# Initialize model
print('Downloading model bundle. This will take less than a minute...')
# you can change it to lookback_rnn or attention_rnn, instead of basic_rnn, as required
note_seq.notebook_utils.download_bundle('basic_rnn.mag', '/content/')
# Import dependencies.
from magenta.models.melody_rnn import melody_rnn_sequence_generator
from magenta.models.shared import sequence_generator_bundle
from note_seq.protobuf import generator_pb2
from note_seq.protobuf import music_pb2
# Initialize the model.
print("Initializing Melody RNN...")
bundle = sequence_generator_bundle.read_bundle_file('/content/basic_rnn.mag')
generator_map = melody_rnn_sequence_generator.get_generator_map()
melody_rnn = generator_map['basic_rnn'](checkpoint=None, bundle=bundle)
melody_rnn.initialize()
print('🎉 Done!')
#Continuing a sequence
#Melody_RNN generates note sequences beyond raga notes
#This is not a desired outcome pattern for raga
# Model options. Change these to get different generated sequences!
input_sequence = malahari # change this to any raga sequence you used earlier, if you want
num_steps = 150 # change this for shorter or longer sequences
temperature = 1.0 # the higher the temperature the more random the sequence.
# Set the start time to begin on the next step after the last note ends.
last_end_time = (max(n.end_time for n in input_sequence.notes)
if input_sequence.notes else 0)
qpm = input_sequence.tempos[0].qpm
seconds_per_step = 60.0 / qpm / melody_rnn.steps_per_quarter
total_seconds = num_steps * seconds_per_step
generator_options = generator_pb2.GeneratorOptions()
generator_options.args['temperature'].float_value = temperature
generate_section = generator_options.generate_sections.add(
start_time=last_end_time + seconds_per_step,
end_time=total_seconds)


In [ ]:
# Ask the model to continue the sequence.
sequence = melody_rnn.generate(input_sequence, generator_options)
note_seq.plot_sequence(sequence)
note_seq.play_sequence(sequence, synth=note_seq.fluidsynth)

## Converting a `NoteSequence` to MIDI

When you called the "play_sequence" method above, this converted the `NoteSequence` to MIDI, and created an HTML widget to play it. This method is specially made for colab notebooks, so it won't work inside your Python script. That method uses the Magenta built-in [conversion methods](https://github.com/magenta/note-seq/blob/main/note_seq/midi_io.py#L51), which you can use in your python script:

In [ ]:
# This creates a file called `drums_sample_output.mid`, containing the drums solo we've been using.
note_seq.sequence_proto_to_midi_file(drums, 'drums_sample_output.mid')

# This is a colab utility method to download that file. In your Python script, you 
# would just write it to disk.
files.download('drums_sample_output.mid')

## Useful helpers
There are a lot of other helper methods sprinkled around the `note_seq` codebase that you might need but not know where to find. Here are some of our favourites:

- [converting](https://github.com/magenta/note-seq/blob/main/note_seq/midi_io.py) between MIDI and NoteSequences
- [trimming, concatenating and expanding](https://github.com/magenta/note-seq/blob/main/note_seq/sequences_lib.py) NoteSequences
- [colab notebook](https://github.com/magenta/note-seq/blob/main/note_seq/notebook_utils.py) utils

# Step 2. Using Machine Learning to make music

`note_seq` has several Machine Learning models, each with different strengths. All models are built with [Tensorflow](https://www.tensorflow.org), so they will run faster if you can run them on a GPU. Here are some of the most popular ones:

- [**MelodyRNN**](https://github.com/magenta/magenta/tree/main/magenta/models/melody_rnn) - you give it a NoteSequence, and it continues it in the style of your original NoteSequence.
- [**MusicVAE**](https://github.com/magenta/magenta/tree/main/magenta/models/music_vae) - generates brand new NoteSequences or interpolates between two sequences.
- [**Onsets and Frames**](https://github.com/magenta/magenta/tree/main/magenta/models/onsets_frames_transcription) -- transcribes piano audio

Now that we know how to use `NoteSequences`, adding some basic Machine Learning is a continuation of that. The pattern for using any of these models is:

- Load `note_seq` (which we already know how to do!)
- Create a model from a downloaded checkpoint (i.e. where the weights, or the encoding, of the model lives)
- Ask the model to do something.

## Melody RNN

A MelodyRNN is an [LSTM-based](https://colah.github.io/posts/2015-08-Understanding-LSTMs/) language model for musical notes -- it is best at continuing a NoteSequence that you give it.

To use it, you need to give it a sequence to continue and the model will return the following sequence.

This example shows how to use the basic Melody RNN model -- check out the [docs](https://github.com/magenta/magenta/tree/main/magenta/models/melody_rnn) for the other models, such as `lookback_rnn` and `attention_rnn`.

### Initialize the model

In [ ]:
print('Downloading model bundle. This will take less than a minute...')
note_seq.notebook_utils.download_bundle('basic_rnn.mag', '/content/')

# Import dependencies.
from magenta.models.melody_rnn import melody_rnn_sequence_generator
from magenta.models.shared import sequence_generator_bundle
from note_seq.protobuf import generator_pb2
from note_seq.protobuf import music_pb2

# Initialize the model.
print("Initializing Melody RNN...")
bundle = sequence_generator_bundle.read_bundle_file('/content/basic_rnn.mag')
generator_map = melody_rnn_sequence_generator.get_generator_map()
melody_rnn = generator_map['basic_rnn'](checkpoint=None, bundle=bundle)
melody_rnn.initialize()

print('🎉 Done!')

### Continuing a sequence

With Melody RNN, you can configure the number of steps the new sequence will be, as well as the "temperature" of the result -- the higher the temperature, the more random (and less like the input) your sequence will be. You can play around with these values and see how the resulting sequences are different:

In [ ]:
# Model options. Change these to get different generated sequences! 

input_sequence = twinkle_twinkle # change this to teapot if you want
num_steps = 128 # change this for shorter or longer sequences
temperature = 1.0 # the higher the temperature the more random the sequence.

# Set the start time to begin on the next step after the last note ends.
last_end_time = (max(n.end_time for n in input_sequence.notes)
                  if input_sequence.notes else 0)
qpm = input_sequence.tempos[0].qpm 
seconds_per_step = 60.0 / qpm / melody_rnn.steps_per_quarter
total_seconds = num_steps * seconds_per_step

generator_options = generator_pb2.GeneratorOptions()
generator_options.args['temperature'].float_value = temperature
generate_section = generator_options.generate_sections.add(
  start_time=last_end_time + seconds_per_step,
  end_time=total_seconds)

# Ask the model to continue the sequence.
sequence = melody_rnn.generate(input_sequence, generator_options)

note_seq.plot_sequence(sequence)
note_seq.play_sequence(sequence, synth=note_seq.fluidsynth)


## Music VAE

A [MusicVAE](https://g.co/magenta/musicvae) is a variational autoencoder made up of an Encoder and Decoder -- you can think of the encoder as trying to summarize all the data you give it, and the decoder as trying to recreate the original data, based on this summarized version. As a generative model, you can think of a VAE as coming up with new sequences that could be a decoding of some summarized version.

The Music VAE implementation in `magenta/music` in particular does two things: it can create new sequences (which are reconstructions or variations of the input data), or it can interpolate between two.

### Initialize the model

In [ ]:
print('Copying checkpoint from GCS. This will take less than a minute...')
# This will download the mel_2bar_big checkpoint. There are more checkpoints that you
# can use with this model, depending on what kind of output you want
# See the list of checkpoints: https://github.com/magenta/magenta/tree/main/magenta/models/music_vae#pre-trained-checkpoints
!gsutil -q -m cp -R gs://download.magenta.tensorflow.org/models/music_vae/colab2/checkpoints/mel_2bar_big.ckpt.* /content/

# Import dependencies.
from magenta.models.music_vae import configs
from magenta.models.music_vae.trained_model import TrainedModel

# Initialize the model.
print("Initializing Music VAE...")
music_vae = TrainedModel(
      configs.CONFIG_MAP['cat-mel_2bar_big'], 
      batch_size=4, 
      checkpoint_dir_or_path='/content/mel_2bar_big.ckpt')

print('🎉 Done!')


### Creating new sequences

With Music VAE, you can configure how many new sequences to generate, the number of steps the new sequence will be, as well as the "temperature" of the result -- the higher the temperature, the more random (and less like the input) your sequence will be. You can play around with these values and see how the resulting sequences are different:

In [ ]:
generated_sequences = music_vae.sample(n=2, length=80, temperature=1.0)

for ns in generated_sequences:
  # print(ns)
  note_seq.plot_sequence(ns)
  note_seq.play_sequence(ns, synth=note_seq.fluidsynth)

### Interpolating between two sequences

In [ ]:
# We're going to interpolate between the Twinkle Twinkle Little Star
# NoteSequence we defined in the first section, and one of the generated
# sequences from the previous VAE example

# How many sequences, including the start and end ones, to generate.
num_steps = 8;

# This gives us a list of sequences.
note_sequences = music_vae.interpolate(
      mg,
      twinkle_twinkle, 
      num_steps=num_steps,
      length=32)

# Concatenate them into one long sequence, with the start and 
# end sequences at each end. 
interp_seq = note_seq.sequences_lib.concatenate_sequences(note_sequences)

note_seq.play_sequence(interp_seq, synth=note_seq.fluidsynth)
note_seq.plot_sequence(interp_seq)

# That's it!

You're now ready to build your own amazing, Machine Learning powered, music instrument! If you want more information, you can check out:

- some [demos](https://magenta.tensorflow.org/demos) `#MadeWithMagenta`
- some more of our [Colab notebooks](https://magenta.tensorflow.org/demos#colab-notebooks)
- the [documentation](https://github.com/magenta/magenta)
- the [Magenta blog](https://magenta.tensorflow.org/blog), which talks about all the mathy bits we skipped.

Have fun! 💕